# Projeto Final - Ciência dos dados
### Alunos:
- Beatriz Lourenço 
- Fernando Kenji 
- Sophia Kerber
- Pedro H. Menezes

<font color='darkcian'>ÍNDICE</font>
___
1. [Dataset](#Dataset)
2. [Pergunta Motivadora](#Pergunta_Motivadora)
3. [Por que regressão linear?](#Por_que_regressão_linear?)
4. [Análise exploratória](#Analise_exploratoria)
5. [Limpeza dos dados e regressão](#Limpeza_dos_dados_e_regressao)
6. [Conclusão](#Conclusão)
7. [R²](#R²)

<a id='Dataset'></a>
## <font color='#FA8072'> O Dataset</font>
___
<font color='darkcian'>Dataset:</font> https://www.kaggle.com/meetnaren/goodreads-best-books

<font color='darkcian'>Objetivo do dataset:</font> o objetivo deste dataset específico é a simples curiosidade. O dataset é facilmente obtido por meio do API do Goodreads, de modo que qualquer um possa fazer análises como a deste projeto. Neste caso, o autor queria saber se era possível prever alguma variável (os exemplos dados foram a avaliação e gênero) pela descrição de um livro.

<a id='Pergunta_Motivadora'></a>
### <font color='#FA8072'> Pergunta Motivadora:</font>
___
Nossa pergunta é se é possível prever a avaliação de um livro a partir do número de páginas, formato do livro (Capa dura, capa de papel, versão Kindle…), inicial do autor, gênero do livro e número de reviews (ou seja, a quantidade de pessoa que comentaram nas suas avaliações).   

<a id='Por_que_regressão_linear?'></a>
### <font color='#FA8072'>Por que regressão linear?</font>
___

Como o objetivo do projeto era prever a avaliação do livro, decidiu-se utilizar a regressão linear para tanto, uma vez que a variável de saída é quantitativa. A ideia da regressão linear é aproximar os dados de uma reta, tentando diminuir o erro quadrático ao máximo. Utilizamos o One Hot Enconder, aliado ao SciKitLearn para dividir o dataset, classificar as variáveis qualitativas em números binários, e finalmente fazer a predição e o teste do dataset.

As técnicas que se prestam a este tipo de análise: regressão linear, regression tree, random forest regression

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import seaborn
import statsmodels.api as sm

In [ ]:
csv = pd.read_csv('book_data.csv')
csv.head()

<a id='Analise_exploratoria'></a>
## <font color='#FA8072'>Análise Exploratória</font>


___
### <font color='CORNFLOWERBLUE'>Para iniciar a análise exploratória vale entender como cada uma das variáveis se correlacionam e de que maneira o book rating em função das outras variáveis do dataframe. Para isso, utilizamos scatterplots para ver a correlação entre variáveis quantitativas X quantitativas; histogramas e boxplots divididos por categorias entre variáveis qualitativas X quantitativas; e crosstabs entre qualitativas X qualitativas.</font>

In [ ]:
plt.scatter(csv.book_rating_count, csv.book_rating, color='b')
seaborn.regplot(x=csv.book_rating_count,y=csv.book_rating)

### <font color='CORNFLOWERBLUE'>Não há visivelmente uma correlação entre a contagem da avaliação do livro e sua própria avaliação, logo elas não estão muito bem correlacionadas e possivelmente usar ela na regressão.

In [ ]:
plt.scatter(csv.book_review_count, csv.book_rating, color='green')
seaborn.regplot(x=csv.book_review_count,y=csv.book_rating)

### <font color='CORNFLOWERBLUE'>Da mesma forma que a anterior as duas variáveis aqui não estão evidentemente correlacionadas e não serão utilizadas para o modelo.</font>

In [ ]:
# Retirando todas as variáveis que não serão utilizadas
csv.drop(['book_isbn','book_desc','book_edition','book_rating_count','image_url'],axis=1,inplace=True)
# Retirando os livros com informações faltando
csv=csv.dropna()
csv.head()

In [ ]:
def limpando(csv):
    # Ajeitando as variáveis
    ar = csv['genres'].tolist()
    ar1 = []
    iniciais = []
    paginas = []

    # Escolhendo como gênero literário apenas o primeiro listado.
    for i in ar:
        if i == "":
            ar1.append("")
        else:
            a = i.split('|')
            ar1.append(a[0])

    # Selecionando apenas a inicial dos nomes dos autores
    for i in csv['book_authors']:
        iniciais.append(i[0])

    # Apenas o número de páginas aparecerá, sem a palavra "pages"
    for i in csv['book_pages']:
        numero = i.split(' ')
        paginas.append(int(numero[0]))
    return ar1,paginas,iniciais

ar1,paginas,iniciais = limpando(csv)
print(ar1[0:10],paginas[0:10],iniciais[0:10])

In [ ]:
# Criando colunas no DataFrame para as iniciais dos autores, os números de páginas e o principal gênero do livro
ar1=pd.DataFrame({'genero':ar1},index = csv.index)
iniciais_ = pd.DataFrame({'iniciais':iniciais}, index = csv.index)
paginas_ = pd.DataFrame({'paginas':paginas}, index = csv.index)

In [ ]:
csv = csv.join(ar1,how='inner')
csv = csv.join(iniciais_,how='inner')
csv = csv.join(paginas_,how='inner')

In [ ]:
csv.head()

In [ ]:
csv['genero'].value_counts()[40:70]

In [ ]:
print('Gêneros mais famosos:')
fantasy = csv.loc[(csv['genero']=='Fantasy')]
print('Média da Fantasia: ' + str(fantasy['book_rating'].mean()))
classics = csv.loc[(csv['genero']=='Classics')]
print('Média do clássico: ' + str(classics['book_rating'].mean()))
fiction = csv.loc[(csv['genero']=='Fiction')]
print('Média da ficção: ' + str(fiction['book_rating'].mean()))
print('-----------------')
print('Gêneros pouco famosos:')
economics = csv.loc[(csv['genero']=='Economics')]
print('Média da economics: ' + str(economics['book_rating'].mean()))
war = csv.loc[(csv['genero']=='War')]
print('Média do gênero Guerra: ' + str(war['book_rating'].mean()))
sports = csv.loc[(csv['genero']=='Sports')]
print('Média do gênero Esportes: ' + str(sports['book_rating'].mean()))


### <font color='CORNFLOWERBLUE'>A partir da análise acima é possível observar que, na média, os gêneros não alteram muito a avaliação do livro. Porém, como o campo amostral ficou pequeno, seria interessante pensar em outra maneira de ver a correlação entre gênero e avaliação do livro</font>

In [ ]:
plt.scatter(csv['genero'].sort_values(), csv.book_rating)
#seaborn.regplot(x=csv['genero'].sort_values(),y=csv.book_rating)

### <font color='CORNFLOWERBLUE'>No gráfico acima, apesar de não se saber bem quais são cada um dos gêneros, pode enxergar que não há uma forte relação entre o gênero e sua avaliação, podendo ser deixado de lado na construção da regressão linear.</font>

In [ ]:
plt.scatter(csv.paginas, csv.book_rating, color='green')
seaborn.regplot(x=csv.paginas,y=csv.book_rating)

### <font color='CORNFLOWERBLUE'>Com o gráfico acima é possível perceber que há uma certa correlação com a avaliação e o número de páginas. Como o gráfico acima está com valores que não permitem analisar o local onde há mais pontos, vamos focar em uma região específica (até 1000 páginas).</font>

In [ ]:
pag = csv.loc[(csv.paginas < 1000)]
pag.book_rating.head()

In [ ]:
plt.scatter(pag.paginas, pag.book_rating, color='green',alpha = 0.3)
seaborn.regplot(x=pag.paginas,y=pag.book_rating, fit_reg=True)

### <font color='CORNFLOWERBLUE'>A partir do gráfico acima é possível perceber que a correlação entre essas variáveis não é tão grande. Portanto, elas podem ser deixadas de serem levadas em conta na hora da regressão neste primeiro momento.</font>

In [ ]:
plt.scatter(csv['iniciais'].sort_values(), csv.book_rating, color='purple',alpha=0.2)
#seaborn.regplot(x=csv['iniciais'].sort_values(),y=csv.book_rating)

### <font color='CORNFLOWERBLUE'>Com o gráfico acima fica complicado de ver as iniciais dos autores pela quantidade que existe, não podendo ser percebido nenhum padrão. Para melhorar a visualização, faremos uma secção do gráfico.</font>

In [ ]:
ini = csv[['iniciais','book_rating']]
inia= ini.sort_values(by='iniciais')[0:47000]

In [ ]:
plt.scatter(inia.iniciais, inia.book_rating, color='purple',alpha = 0.1)

### <font color='CORNFLOWERBLUE'>Com o gráfico acima é possível perceber que algumas letras ficam mais na média dos valores (4) e algumas conseguem se deslocar mais para regiões da parte de cima quanto da parte de baixo. Logo, a variável inicial do nome do autor pode ser interessante de ser analisada na regressão.</font>

<a id='Limpeza_dos_dados_e_regressao'></a>
## <font color='#FA8072'>Limpeza dos dados e regressão</font>
___

Decidiu-se tirar e modificar algumas variáveis do código, para que o resultado da regressão fosse otimizado. Para isso, 
o código foi modificado aos poucos de acordo com o [R²](#R²). Para melhorar os resultados, decidimos tirar os nomes dos autores e substituí-los pelas iniciais dos mesmos. Além disso, decidiu-se trocar os vários gêneros por somente um

In [ ]:
# Retirando as colunas irrelevantes para a análise geral de avaliações
csv.drop(['book_authors','genres','book_pages'],axis=1,inplace=True)

In [ ]:
csv.head()

In [ ]:
# Separando as variáveis em quantitativas, qualitativas e a desejada
y = csv['book_rating'].values
num = csv[['paginas','book_review_count']]
qual = csv[['book_format','genero','iniciais']]

In [ ]:
qual.head()

In [ ]:
encoder = OneHotEncoder()
capa_encoded = encoder.fit_transform(qual)

In [ ]:
x_num = num.values

In [ ]:
#Transformar em array facilita a exploração dos dados
x_encoded = capa_encoded.toarray()

In [ ]:
#Juntando a lista binária e as páginas
X = np.concatenate((x_num,x_encoded), axis=1)

In [ ]:
X[0]

In [ ]:
#Rating do livro 1
y[0]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing,utils, metrics
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

for i in range(len(y_pred)):
    if y_pred[i]>5:
        y_pred[i]=5
    elif y_pred[i]<0:
        y_pred[i]=0
        
df = pd.DataFrame({'Atual': y_test.flatten(), 'Predito': y_pred.flatten()})
df.head()

In [ ]:
df1 = df.head(25)
df1.plot(kind='bar',figsize=(16,10))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.xlabel('Book Index')
plt.ylabel('Rating')
plt.show()

In [ ]:
print("O R² é " + str(np.sqrt(metrics.mean_squared_error(df['Atual'],df['Predito']))))

In [ ]:
plt.hist(df['Atual'] - df['Predito'],bins=100,density=True);
plt.grid(True)

## <font color='#FA8072'>Conclusão</font>
<a id='Conclusão'></a>
___

A partir das métricas acima (sobretudo do R²) é possível observar que o modelo dista das medidas que de fato existem em até uma unidade (tanto para mais, quanto para menos). Visto que as avaliações vão de 0 até 5 e que nosso R² é 0,32, temos que ele tem um erro relativamente pequeno.  Porém, para  ver de que maneira poderíamos ajustar o modelo para ficar da melhor maneira, podemos refazer a análise, contudo, ao invés de usar valores inteiros, vamos fazer uma [`classificação`](classification.ipynb) dos livros.

## <font color='#FA8072'>R²</font>

___
<a id='R²'></a>

### Como melhorar a análise?
___

Pode-se melhorar a análise feita por meio da K-Folds Cross Validation.